## Topics Recomposer

In [1]:
import pickle as pkl
import numpy as np

In [8]:
topics = pkl.load(open("/scratch/hartley/twitter_covid_insights/insights_All/topics_700_occ400.pkl", 'rb'))
old_topics = pkl.load(open('/scratch/massemin/twitter_covid_insights_v3/insights_All/topics.pkl', 'rb'))
new_hashes = set([x for topic_nb in topics[0].keys() for x in topics[0][topic_nb]])
recomposed_path = "/scratch/hartley/twitter_covid_insights/insights_All/recomposed_topics.pkl"

# Look for overlap with current topics, add if none
to_add = []
for t in old_topics[0] :
    found = False
    for h in old_topics[0][t] :
        if h in new_hashes :
            found=True
    if not found :
        to_add.append(t)

t_mapping = dict(zip(to_add, np.arange(len(to_add))+len(topics[0])))
recomposed_topics = dict(topics[0])

# Add old topics
for m in t_mapping :
    recomposed_topics[t_mapping[m]] = old_topics[0][m]
    
recomposed_topics = (recomposed_topics, topics[1])
pkl.dump(recomposed_topics, open(recomposed_path, 'wb'), protocol=4)

## Merge Country files

In [117]:
import pycountry

__MACOSX/                  final_data.csv                 vaccinations/
countries/                 final_data_generator.ipynb     vaccinations.csv
economic_df_builder.ipynb  swissre/                       weather/
economic_weighted.csv      unemp_daily_with_forecast.csv
economic_weighted_2.csv    vaccination_percentages.csv


In [200]:
d = pd.read_csv('/scratch/hartley/WhatIf/daily_country_data.csv')

/scratch/hartley/rapids-21.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (68,69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [201]:
d.columns

Index(['country_id', 'date', 'name', 'cases', 'deaths', 'recovered',
       'new_cases', 'new_deaths', 'total_population_all',
       'total_population_below_1', 'total_population_1_to_4',
       'total_population_5_to_14', 'total_population_15_to_24',
       'total_population_25_to_34', 'total_population_35_to_54',
       'total_population_55_to_74', 'total_population_above_75',
       'population_percentage_over_60', 'hospital_beds_per_1000',
       'physicians_per_1000', 'nurses_per_1000', 'fatality_rate_percent',
       'cases_per_100000', 'deaths_per_100000', 'cases_per_1000_hospital_beds',
       'deaths_per_1000_hospital_beds', 'cases_per_1000_nurses',
       'deaths_per_1000_nurses', 'cases_per_1000_physicians',
       'deaths_per_1000_physicians', 'days_since_outbreak',
       'new_cases_percent_increase', 'avg_3d_new_cases_percent_increase',
       'unemployment_rate_2019_03_31', 'unemployment_rate_2019_06_30',
       'unemployment_rate_2019_09_30', 'unemployment_rate_2019_12

In [ ]:
pkl.load(open())

In [195]:
# Load epi data
columns=['country_id', 'date', 'cases', 'deaths', 'recovered', 'new_cases', 'mobility_google_index', 'tests']
new_countries = pd.read_csv('/scratch/hartley/WhatIf/daily_country_data.csv', usecols=columns)
new_countries = new_countries[~new_countries.country_id.isna()]

# Load vaccine data
cols = ['iso_code', 'date', 'people_vaccinated', 'daily_vaccinations']
new_vaccines = pd.read_csv('/scratch/hartley/WhatIf/vaccinations.csv', usecols=cols)

# Merge both db
new_countries['iso_code'] = new_countries.country_id.apply(lambda n : pycountry.countries.get(alpha_2=n).alpha_3)
merged = pd.merge(left=new_countries, right=new_vaccines, how='outer',on=['iso_code', 'date']).drop('country_id', axis=1)
merged = merged[merged.date <= '2021-09-30']

merged.to_csv('/scratch/hartley/WhatIf/final_epi_file.csv')

In [197]:
merged

,date,cases,deaths,recovered,new_cases,mobility_google_index,tests,iso_code,people_vaccinated,daily_vaccinations
0,2020-02-03,1.0,0.0,1.0,1.0,NaN,NaN,BEL,NaN,NaN
1,2020-02-04,1.0,0.0,1.0,0.0,NaN,NaN,BEL,NaN,NaN
2,2020-02-05,1.0,0.0,1.0,0.0,NaN,NaN,BEL,NaN,NaN
3,2020-02-06,1.0,0.0,1.0,0.0,NaN,NaN,BEL,NaN,NaN
4,2020-02-07,1.0,0.0,1.0,0.0,NaN,NaN,BEL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
134015,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,ZWE,3036623.0,31405.0
134016,2021-09-27,NaN,NaN,NaN,NaN,NaN,NaN,ZWE,3050981.0,31282.0
134017,2021-09-28,NaN,NaN,NaN,NaN,NaN,NaN,ZWE,3060086.0,29469.0
134018,2021-09-29,NaN,NaN,NaN,NaN,NaN,NaN,ZWE,3071643.0,28149.0


In [2]:
import cupy as cp
from cuml.metrics import pairwise_distances

In [26]:
import numpy as np
from tqdm import tqdm

In [28]:
np.array([[1, 2], [3, 4], [5, 6]]).mean(axis=0)

array([3., 4.])

In [35]:
arr = np.arange(1000).reshape((200, 5)).astype(float)
batch_nb = 20

# Chunks to be computed
splits = (list(np.arange(len(arr))) + [len(arr)])[::batch_nb]
splits_tup = list(zip(splits, splits[1:]))


for e, s in tqdm(enumerate(splits_tup)) :
    sub = arr[s[0]:s[1]]
    for i in sub :
        X = cp.array(sub.transpose())        
        dst = pairwise_distances(X, metric='euclidean')

10it [00:01,  5.30it/s]


In [38]:
a = [[1, 2], [3, 4]]
b = [[5, 2], [3, 4]]
c = [[1, 2], [3, 8]]
d = [[1, 2], [3, 20]]

In [43]:
np.hstack((np.vstack((a, b)), np.vstack((c, d))))

array([[ 1,  2,  1,  2],
       [ 3,  4,  3,  8],
       [ 5,  2,  1,  2],
       [ 3,  4,  3, 20]])

In [37]:
dst

array([[ 0.        ,  4.47213595,  8.94427191, 13.41640786, 17.88854382],
       [ 4.47213595,  0.        ,  4.47213595,  8.94427191, 13.41640786],
       [ 8.94427191,  4.47213595,  0.        ,  4.47213595,  8.94427191],
       [13.41640786,  8.94427191,  4.47213595,  0.        ,  4.47213595],
       [17.88854382, 13.41640786,  8.94427191,  4.47213595,  0.        ]])

In [21]:
splits

[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]

In [22]:
splits_tup

[(0, 20),
 (20, 40),
 (40, 60),
 (60, 80),
 (80, 100),
 (100, 120),
 (120, 140),
 (140, 160),
 (160, 180),
 (180, 200)]

In [39]:
X = cp.array([[2.0, 3.0], [3.0, 5.0], [5.0, 8.0]])

dst = pairwise_distances(X, metric='euclidean')

In [40]:
1 - dst

array([[ 1.        , -1.23606798, -4.83095189],
       [-1.23606798,  1.        , -2.60555128],
       [-4.83095189, -2.60555128,  1.        ]])

In [37]:
type(1 - dst)

cupy._core.core.ndarray

In [29]:
cosine_similarity([[1, 2, 3, 4], [5, 6, 7, 8], [1, 2, 3, 4]])

array([[1.        , 0.96886393, 1.        ],
       [0.96886393, 1.        , 0.96886393],
       [1.        , 0.96886393, 1.        ]])

In [37]:
import pickle as pkl

In [3]:
trends = pkl.load(open("/scratch/hartley/tweets_piped twitter_covid_insights/insights_All/trends_recomposed.pkl", 'rb'))

In [ ]:
tuple(topic_id -> array(occs), metatopic_id -> array(occ))

In [5]:
trends[1].ke

72

In [38]:
#topics_growth = pkl.load(open("/scratch/hartley/twitter_covid_insights/insights_All/topics_growth.pkl", 'rb'))
topics = pkl.load(open("/scratch/hartley/twitter_covid_insights/insights_All/topics_700_occ400.pkl", 'rb'))

In [ ]:
mt_id -> list(topic_id)

In [6]:
topics[1]

NameError: name 'topics' is not defined

In [22]:
d = pkl.load(open('/scratch/hartley/twitter_covid_insights/insights_All/s2v_dict.pkl', 'rb'))

In [24]:
d.keys()

dict_keys(['#reconstruyamoslahipica', '#sbot'])

In [20]:
import numpy as np
np.array('1 2 3 4'.split(' ')).astype(float)

array([1., 2., 3., 4.])

In [17]:
topics[0]

{0: array(['#コロナに負けるな', '#おうち時間', '#ステイホーム', '#ダイエット', '#テイクアウト', '#外出自粛',
        '#ランチ', '#筋トレ', '#居酒屋'], dtype='<U37'),
 1: array(['#東京五輪の中止を求めます', '#自民党に殺される', '#東京五輪の開催中止を求めます', '#投票倍増委員会',
        '#スガやめろ', '#五輪中止の決断を', '#スガ政権の退陣を求めます', '#五輪やってる場合じゃない',
        '#もううんざりだよ自民党', '#オリンピックより命を守れ', '#菅やめろ', '#東京五輪の即時中止を求めます',
        '#自民党に政権担当能力はありません', '#日本のコロナは自民党の人災', '#自民党政権が日本を滅ぼす',
        '#小池百合子に殺される', '#選挙に行こう', '#自民党は全員落とす', '#こんなひどい政治は初めてだ', '#政権交代',
        '#自民党の解党を求めます', '#五輪やめて命まもれ', '#自民党は利権と汚職と税金泥棒', '#大阪のコロナは維新の人災',
        '#コロナ犠牲者激増はスガの人災', '#野党共闘で政権交代', '#自民党政治を終わらせよう',
        '#自民党のしたこと絶対に忘れない', '#消費税廃止', '#菅政権の退陣を求めます', '#菅政権に殺される', '#菅辞めろ',
        '#もういらないだろ自民党', '#菅義偉に殺される', '#維新に殺される', '#政権交代しかない',
        '#小池百合子に騙されるな', '#維新はいらない', '#菅に殺される', '#自民公明維新には絶対投票しない',
        '#維新は本当にいらない', '#自民党には投票しない'], dtype='<U37'),
 2: array(['#hypocrisy', '#hypocrite', '#hypocrites', '#liar', '#traitor',
        '#liars', '#traitors'], dtype='<U37'),
 3: array(['#felizl

In [ ]:
topics = pkl.load('/')

In [2]:
########### Topics creation ############
if topics_computed and not force_compute :
    print("Step 3/5 : Topics were already computed")
else :
    print(f"Step 3/5 : Computing topics, dumping in {topics_path}")
    find_topics(model, word_counts, topics_path, max_absorption=100, min_clust_size=5, growth_path=growth_path)

Step 3/5 : Computing topics, dumping in /scratch/hartley/twitter_covid_insights_small/insights_All/topics.pkl


100%|██████████| 500/500 [00:54<00:00,  9.23it/s]


Exception: Not enough data to find significant topics

In [24]:
import nltk

In [27]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hartley/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [42]:
model.wv.

TypeError: 'numpy.ndarray' object is not callable

In [39]:
model.wv.vectors.transpose().shape

(300, 3371)

In [3]:
import cudf
from cuml.cluster import DBSCAN

# Create and populate a GPU DataFrame
x = cudf.DataFrame()
x['0'] = [1.0, 2.0, 5.0, 4]
x['1'] = [2.0, 2.0, 1.0, 4]
# Setup and fit clusters
x = DBSCAN(eps=1.0, min_samples=1).fit(x)
print(x.labels_)

0    0
1    0
2    1
3    2
dtype: int32


In [11]:
dbscan_float

DBSCAN()

In [6]:
conda env list

# conda environments:
#
base                     /opt/conda
                      *  /scratch/hartley/rapids-21.10


Note: you may need to restart the kernel to use updated packages.


In [8]:
conda activate /scratch/hartley/rapids-21.10


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [9]:
conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
absl-py                   0.12.0                   pypi_0    pypi
appdirs                   1.4.4                    pypi_0    pypi
argon2-cffi               20.1.0           py38h27cfd23_1  
astroid                   2.5.2                    pypi_0    pypi
astunparse                1.6.3                    pypi_0    pypi
async_generator           1.10               pyhd3eb1b0_0  
attrs                     20.3.0             pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
black                     20.8b1                   pypi_0    pypi
blas                      1.0                         mkl  
bleach                    3.3.0              pyhd3eb1b0_0  
brotlipy                  0.7.0           py38h27cfd23_1003  
ca-certificates           2021.1.19            h06a4308_1  
cachetools        


Note: you may need to restart the kernel to use updated packages.
